# Compare Toronto Neighborhood Data to Manhattan NY

Manhattan is one of the largest and diverse boroughs in the city of New York. Given its size it can be called a city unto its own. In this project we will treat it as a city and compare it to Canadas largest city Toronto. The aim of this exercise is to determine if neighborhoods in Toronto have facility patterns like Manhattan. In other words, we will look to see if they cluster similarly.

We will take Toronto Data listing neighborhoods and use Foursquare location data to find the types of establishments in the area. We will then cluster the neighborhoods into 5 clusters based on these establishment data and store the cluster. Next, we will do the same for Manhattan data and build a k-Means clustering model using the Manhattan data for the fit (training.)

Finally, we will use the actual cluster input and output data as the testing data into the k-mean model built from the Manhattan data. Based on the accuracy for the model prediction we will make decisions about how similar Toronto is to Manhattan regarding establishments.

Data from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M will be used to create Toronto neighborhood data and actual clustering. The same data will be used as test data against the Mannahttan (fit) model to determine accuracy. 
Data from  https://cocl.us/new_york_dataset will be used to train the Mannhattan model for later use testing Toronto data.

In [ ]:
# importing necessary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

In [3]:
# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [4]:
import pandas
import requests
from bs4 import BeautifulSoup
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'html')

table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pandas.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

#df.head(15)

In [5]:
df.shape

(180, 3)

In [6]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
df.shape

(103, 3)

In [7]:
df.drop(df[df['Neighbourhood']=="Not assigned"].index,axis=0, inplace=True)
df.shape

(103, 3)

In [8]:
df = df.reset_index()
df.head()

,index,PostalCode,Borough,Neighbourhood
0,3,M3A,North York,Parkwoods
1,4,M4A,North York,Victoria Village
2,5,M5A,Downtown Toronto,Regent Park / Harbourfront
3,6,M6A,North York,Lawrence Manor / Lawrence Heights
4,7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [9]:
df = df.groupby('PostalCode').agg(lambda x: ','.join(x)).reset_index()
df.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [10]:
df['City'] = 'Toronto'

In [11]:
df['Country'] = 'Canada'

In [12]:
df.shape

(103, 5)

In [13]:
df['Neighbourhood']= df['Neighbourhood'].str.replace("/", " , ")
#df['Neighbourhood'] = (df['Neighbourhood'].str.split(",",expand=True)[0].reset_index(drop=True))
df.head()

,PostalCode,Borough,Neighbourhood,City,Country
0,M1B,Scarborough,"Malvern , Rouge",Toronto,Canada
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",Toronto,Canada
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",Toronto,Canada
3,M1G,Scarborough,Woburn,Toronto,Canada
4,M1H,Scarborough,Cedarbrae,Toronto,Canada


In [14]:
!pip install geopy

In [15]:
import pandas as pd

In [16]:
df_geopy = df.reset_index(drop=True)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,City,Country
0,M1B,Scarborough,"Malvern , Rouge",Toronto,Canada
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",Toronto,Canada
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",Toronto,Canada
3,M1G,Scarborough,Woburn,Toronto,Canada
4,M1H,Scarborough,Cedarbrae,Toronto,Canada


In [221]:
#df_geopy['Neighbourhood'] = (df_geopy['Neighbourhood'].str.split(",",expand=True)[0].reset_index(drop=True))
#df_geopy.shape

In [17]:
df_geopy['address'] = df_geopy[['City', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy.head()

,PostalCode,Borough,Neighbourhood,City,Country,address
0,M1B,Scarborough,"Malvern , Rouge",Toronto,Canada,"Toronto, Scarborough, Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",Toronto,Canada,"Toronto, Scarborough, Rouge Hill , Port Unio..."
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",Toronto,Canada,"Toronto, Scarborough, Guildwood , Morningsid..."
3,M1G,Scarborough,Woburn,Toronto,Canada,"Toronto, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,Toronto,Canada,"Toronto, Scarborough, Cedarbrae"


In [18]:
from geopy.geocoders import Nominatim
nom = Nominatim(user_agent='myGeocoder')

In [19]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,City,Country,address,Coordinates
0,M1B,Scarborough,"Malvern , Rouge",Toronto,Canada,"Toronto, Scarborough, Malvern , Rouge","(Baton Rouge, Progress Avenue, Scarborough, Sc..."
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",Toronto,Canada,"Toronto, Scarborough, Rouge Hill , Port Unio...",None
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",Toronto,Canada,"Toronto, Scarborough, Guildwood , Morningsid...","(Toronto Pan Am Sports Centre (TPASC), Morning..."
3,M1G,Scarborough,Woburn,Toronto,Canada,"Toronto, Scarborough, Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,Toronto,Canada,"Toronto, Scarborough, Cedarbrae","(Cedarbrae Collegiate Institute, Markham Road,..."


In [20]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,City,Country,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Malvern , Rouge",Toronto,Canada,"Toronto, Scarborough, Malvern , Rouge","(Baton Rouge, Progress Avenue, Scarborough, Sc...",43.779300,-79.257126
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",Toronto,Canada,"Toronto, Scarborough, Rouge Hill , Port Unio...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",Toronto,Canada,"Toronto, Scarborough, Guildwood , Morningsid...","(Toronto Pan Am Sports Centre (TPASC), Morning...",43.790672,-79.193646
3,M1G,Scarborough,Woburn,Toronto,Canada,"Toronto, Scarborough, Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,Toronto,Canada,"Toronto, Scarborough, Cedarbrae","(Cedarbrae Collegiate Institute, Markham Road,...",43.756467,-79.226692


In [21]:
df_geopy.shape

(103, 9)

In [22]:
df_geopy = df_geopy.dropna()
df_geopy.shape

(52, 9)

In [23]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)
# instantiate the dataframe
neighborhoods['Borough']      = df_geopy['Borough']
neighborhoods['Neighborhood'] = df_geopy['Neighbourhood']
neighborhoods['Latitude']     = df_geopy['latitude']
neighborhoods['Longitude']    = df_geopy['longitude']

In [228]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Scarborough,"Malvern , Rouge",43.779300,-79.257126
2,Scarborough,"Guildwood , Morningside , West Hill",43.790672,-79.193646
3,Scarborough,Woburn,43.759824,-79.225291
4,Scarborough,Cedarbrae,43.756467,-79.226692
5,Scarborough,Scarborough Village,43.743742,-79.211632


In [24]:
neighborhoods.shape

(52, 4)

In [25]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [26]:
CLIENT_ID = 'JUCYNMFC2U2Q4HTU1ERJEPZDZH5FPSJO1PDWELC0QT3G1TFD' # your Foursquare ID
CLIENT_SECRET = 'Y3ERB1SJDQPFEIFOEW2C41RMEBMDMQDBDWKTAJFMJAYHXDQ3' # your Foursquare Secret
VERSION = '20200420' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JUCYNMFC2U2Q4HTU1ERJEPZDZH5FPSJO1PDWELC0QT3G1TFD
CLIENT_SECRET:Y3ERB1SJDQPFEIFOEW2C41RMEBMDMQDBDWKTAJFMJAYHXDQ3


In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
# type your answer here

Toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )


Malvern  ,  Rouge
Guildwood  ,  Morningside  ,  West Hill
Woburn
Cedarbrae
Scarborough Village
Wexford  ,  Maryvale
Agincourt
Steeles West  ,  L'Amoreaux West
Upper Rouge
Hillcrest Village
Bayview Village
Willowdale  ,  Newtonbrook
Willowdale
York Mills West
Willowdale
Parkwoods
Don Mills
Don Mills
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill  ,  Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West  ,  Riverdale
Lawrence Park
Davisville North
Davisville
Moore Park  ,  Summerhill East
Rosedale
Church and Wellesley
Garden District, Ryerson
Central Bay Street
Roselawn
Forest Hill North & West
Glencairn
Humewood-Cedarvale
Dufferin  ,  Dovercourt Village
Runnymede  ,  The Junction North
Parkdale  ,  Roncesvalles
Alderwood  ,  Long Branch
Islington Avenue
Humber Summit
Humberlea  ,  Emery
Weston
Westmount
Northwest


In [30]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

#Toronto_onehot.head()

In [31]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
#Toronto_groupedcvs = Toronto_grouped

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Toronto_venues_sorted = pd.DataFrame(columns=columns)
Toronto_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    Toronto_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

Toronto_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Hong Kong Restaurant,Food Court,Korean Restaurant,Asian Restaurant,Train Station,Cantonese Restaurant,Coffee Shop,Vietnamese Restaurant,Dance Studio
1,"Alderwood , Long Branch",Pizza Place,Coffee Shop,Skating Rink,Pub,Sandwich Place,Pharmacy,Gym,Greek Restaurant,Gourmet Shop,Electronics Store
2,Bayview Village,Bank,Fast Food Restaurant,Gas Station,Persian Restaurant,Sporting Goods Shop,Fish Market,Sandwich Place,Metro Station,Breakfast Spot,Clothing Store
3,Cedarbrae,Fast Food Restaurant,Coffee Shop,Pharmacy,Optical Shop,Shopping Mall,Sandwich Place,Clothing Store,Pizza Place,Park,Paper / Office Supplies Store
4,Central Bay Street,Coffee Shop,Sandwich Place,Gastropub,Japanese Restaurant,Sushi Restaurant,Diner,Spa,Park,Burrito Place,Café


In [34]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 0, 0, 0, 1, 1, 1, 0])

In [36]:
# add clustering labels
Toronto_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(Toronto_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern , Rouge",43.779300,-79.257126,1,Clothing Store,Restaurant,Women's Store,Cosmetics Shop,Bakery,Tea Room,Big Box Store,Shopping Mall,Bookstore,Boutique
2,Scarborough,"Guildwood , Morningside , West Hill",43.790672,-79.193646,1,Athletics & Sports,Curling Ice,Pub,Fried Chicken Joint,Women's Store,Distribution Center,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
3,Scarborough,Woburn,43.759824,-79.225291,0,Fast Food Restaurant,Coffee Shop,Bank,Discount Store,Optical Shop,Pharmacy,Hardware Store,Pizza Place,Clothing Store,Department Store
4,Scarborough,Cedarbrae,43.756467,-79.226692,0,Fast Food Restaurant,Coffee Shop,Pharmacy,Optical Shop,Shopping Mall,Sandwich Place,Clothing Store,Pizza Place,Park,Paper / Office Supplies Store
5,Scarborough,Scarborough Village,43.743742,-79.211632,0,Coffee Shop,Fast Food Restaurant,Shopping Mall,Chinese Restaurant,Pub,Pharmacy,Gym,Greek Restaurant,Ethiopian Restaurant,Electronics Store


In [39]:
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hong Kong Restaurant,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Moving Target,Music Venue,Nail Salon,New American Restaurant,Nightclub,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Women's Store
0,Agincourt,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.333333,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.000000,0.0,

In [63]:
Toronto_Neigh_Clust = Toronto_merged.groupby('Neighborhood').count().reset_index()

In [64]:
Toronto_Neigh_Clust.head()


,Neighborhood,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,"Alderwood , Long Branch",1,1,1,1,1,1,1,1,1,1,1,1,1,1
2,Bayview Village,1,1,1,1,1,1,1,1,1,1,1,1,1,1
3,Cedarbrae,1,1,1,1,1,1,1,1,1,1,1,1,1,1
4,Central Bay Street,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [68]:
Toronto_Neigh_Clust.drop(['Borough', 'Latitude', 'Longitude', '1st Most Common Venue', '2nd Most Common Venue', '3rd Most Common Venue', '4th Most Common Venue', '5th Most Common Venue', '6th Most Common Venue', '7th Most Common Venue', '8th Most Common Venue', '9th Most Common Venue', '10th Most Common Venue'], axis=1, inplace = True)
Toronto_Neigh_Clust.head()

,Neighborhood,Cluster Labels
0,Agincourt,1
1,"Alderwood , Long Branch",1
2,Bayview Village,1
3,Cedarbrae,1
4,Central Bay Street,1


In [72]:
Toronto_Neigh_Clust = Toronto_Neigh_Clust.join(Toronto_grouped.set_index('Neighborhood'), on='Neighborhood')
Toronto_Neigh_Clust.head()

,Neighborhood,Cluster Labels,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Beer Bar,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,College Rec Center,Comfort Food Restaurant,Comic Shop,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hardware Store,Health Food Store,Historic Site,Hobby Shop,Hong Kong Restaurant,Hotel,Housing Development,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Laundromat,Laundry Service,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Movie Theater,Moving Target,Music Venue,Nail Salon,New American Restaurant,Nightclub,Optical Shop,Other Great Outdoors,Outdoor Supply Store,Paintball Field,Paper / Office Supplies Store,Park,Persian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Soccer Field,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Tanning Salon,Tattoo Parlor,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tree,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Women's Store
0,Agincourt,1,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,0.333333,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.083333,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.083333,0.0,0.0,